In [16]:
!pip install flask
!pip install flask_restful

You should consider upgrading via the '/Users/mihira/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 52 kB 3.2 MB/s eta 0:00:011
You should consider upgrading via the '/Users/mihira/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [17]:
import text2emotion as te
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize
import string
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from flask import Flask
from flask_restful import Resource, Api, reqparse
import ast

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mihira/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [19]:
app = Flask(__name__)
api = Api(app)

In [20]:
class MoodAnalysis(Resource):
    @staticmethod
    def t2e_entry_analysis(text):
        mood = te.get_emotion(text)
        return mood
    
    @staticmethod
    def t2e_sent_analysis(text):
        sent_tokenized = {}
        sent_tokenized = nltk.tokenize.sent_tokenize(text)
        mood_list = []
        for sentence in sent_tokenized:
            mood = te.get_emotion(sentence)
            mood_list.append(mood)
        return mood_list
    
    @staticmethod
    def polarity_entry_analysis(text):
        sia = SentimentIntensityAnalyzer()
        mood = sia.polarity_scores(text)
        mood['cummulative'] = TextBlob(text).sentiment.polarity
        return mood
    
    @staticmethod
    def polarity_sent_analysis(text):
        sia = SentimentIntensityAnalyzer()
        sent_tokenized = {}
        sent_tokenized = nltk.tokenize.sent_tokenize(text)
        mood_list = []
        for sentence in sent_tokenized:
            mood = sia.polarity_scores(sentence)
            mood['cummulative'] = TextBlob(sentence).sentiment.polarity
            mood_list.append(mood)
        return mood_list
    
    def post(self):
        parser = reqparse.RequestParser()  # initialize
        
        parser.add_argument('text', required=True)  # add args
        
        args = parser.parse_args()  # parse arguments to dictionary
        
        text = args['text']
        
        mood_analysis_dict = {'t2e_entry_analysis': {}, 't2e_sent_analysis':{}, 'polarity_entry_analysis':{}, 'polarity_sent_analysis':{}}
        
        mood_analysis_dict['t2e_entry_analysis'] = MoodAnalysis.t2e_entry_analysis(text)
        
        mood_analysis_dict['t2e_sent_analysis'] = MoodAnalysis.t2e_sent_analysis(text)
        
        mood_analysis_dict['polarity_entry_analysis'] = MoodAnalysis.polarity_entry_analysis(text)
        
        mood_analysis_dict['polarity_sent_analysis'] = MoodAnalysis.polarity_sent_analysis(text)
        
        return {'data': mood_analysis_dict}, 200

api.add_resource(MoodAnalysis, '/moodanalysis') 

In [ ]:
if __name__ == '__main__':
    app.run()  # run our Flask app

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Nov/2021 17:07:54] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [03/Nov/2021 17:07:54] "GET /favicon.ico HTTP/1.1" 404 -
